# June 18 Neural Network Testing

In [1]:
# Torch 
import torch
import torch.nn as nn

# MNIST1D 
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d 
from mnist1d.data import make_dataset, get_dataset_args

# Custom Modules 
import sys 
sys.path.append('../')
from Conv1d_NN import Conv1d_NN

from models import * 
from train import *

device = 'cpu'

In [2]:
# Conv1d_NN
model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
)

# Get summary 
from torchsummary import summary
summary(model, (1, 40), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 40]             100
         Conv1d_NN-2               [-1, 25, 40]               0
              ReLU-3               [-1, 25, 40]               0
            Conv1d-4               [-1, 25, 40]           1,900
         Conv1d_NN-5               [-1, 25, 40]               0
              ReLU-6               [-1, 25, 40]               0
            Conv1d-7               [-1, 25, 40]           1,900
         Conv1d_NN-8               [-1, 25, 40]               0
              ReLU-9               [-1, 25, 40]               0
          Flatten-10                 [-1, 1000]               0
           Linear-11                   [-1, 10]          10,010
Total params: 13,910
Trainable params: 13,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [3]:
import torch
from torch import nn
# Sequential version of MNIST model 
model1 = nn.Sequential(
   nn.Conv1d(1, 25, 5, 1, 2), 
   nn.ReLU(),
   nn.Conv1d(25, 25, 5, 1, 2),
   nn.ReLU(),
   nn.Conv1d(25, 1, 5, 1, 2),
   nn.ReLU()
)
print(model1.parameters())
# Get summary 
from torchsummary import summary
summary(model1, (1, 40))


<generator object Module.parameters at 0x14f8f3140>
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 40]             150
              ReLU-2               [-1, 25, 40]               0
            Conv1d-3               [-1, 25, 40]           3,150
              ReLU-4               [-1, 25, 40]               0
            Conv1d-5                [-1, 1, 40]             126
              ReLU-6                [-1, 1, 40]               0
Total params: 3,426
Trainable params: 3,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.01
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [4]:
# NN.Module version of MNIST model
model2 = ConvBase_v2(10)

summary(model2, (1, 40), device=device)

Initialized ConvBase model with 5210 parameters
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
            Conv1d-2               [-1, 25, 10]           1,900
            Conv1d-3                [-1, 25, 5]           1,900
           Flatten-4                  [-1, 125]               0
            Linear-5                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


In [5]:
# Model from MNIST1D repository 
model3 = ConvBase(10)

# Get summary
from torchsummary import summary
summary(model2, (1, 40), device=device)

Initialized ConvBase model with 5210 parameters
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
            Conv1d-2               [-1, 25, 10]           1,900
            Conv1d-3                [-1, 25, 5]           1,900
           Flatten-4                  [-1, 125]               0
            Linear-5                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


### Gather data MNIST1D

In [6]:

from mnist1d.data import make_dataset, get_dataset_args

args = get_dataset_args(as_dict=False)


set_seed(args.seed)
args.shuffle_seq = False
data = make_dataset(args = args) # Make the dataset 
data['x'] = torch.Tensor(data['x']).unsqueeze(1)
data['x_test'] = torch.Tensor(data['x_test']).unsqueeze(1)

print(data['x'].shape, data['y'].shape, data['t'].shape)

torch.Size([4000, 1, 40]) (4000,) (40,)


### Train Models

In [7]:

args = get_model_args()
args.device = 'mps'
args.total_steps = 6000

set_seed(args.seed)

print("Custom Conv1d_NN")
results =train_model(data, model, args)
print()

print("Sequential MNIST Model")
results1 = train_model(data, model1, args)
print() 

print("NN.Module MNIST Model")
results2 = train_model(data, model2, args)
print()

print("MNIST1D Model")
results3 = train_model(data, model3, args)
print()


Custom Conv1d_NN
step 1000, dt 18.46s, train_loss 9.205e-01, test_loss 1.460e+00, train_acc 66.3, test_acc 48.1
step 2000, dt 16.45s, train_loss 6.508e-01, test_loss 1.710e+00, train_acc 69.9, test_acc 47.8
step 3000, dt 17.22s, train_loss 6.880e-01, test_loss 1.804e+00, train_acc 72.7, test_acc 47.2
step 4000, dt 16.87s, train_loss 6.980e-01, test_loss 2.088e+00, train_acc 71.5, test_acc 44.8
step 5000, dt 15.42s, train_loss 6.172e-01, test_loss 2.145e+00, train_acc 71.4, test_acc 45.1
step 6000, dt 15.12s, train_loss 5.760e-01, test_loss 2.034e+00, train_acc 72.0, test_acc 44.2

Sequential MNIST Model


RuntimeError: Expected target size [100, 40], got [100]

### Try with less noise data

In [ ]:

from mnist1d.data import make_dataset, get_dataset_args

less_noise_args = get_dataset_args(as_dict=False)
# {'num_samples': 5000, 'train_split': 0.8, 'template_len': 12, 
# 'padding': [36, 60], 'scale_coeff': 0.4, 'max_translation': 48, 
# 'corr_noise_scale': 0.25, 'iid_noise_scale': 0.02, 'shear_scale': 0.75, 
# 'shuffle_seq': False, 'final_seq_length': 40, 'seed': 42, 
# 'url': 'https://github.com/greydanus/mnist1d/raw/master/mnist1d_data.pkl'}

set_seed(less_noise_args.seed)
less_noise_args.shuffle_seq = False
less_noise_args.corr_noise_scale = 0.00w
less_noise_args.iid_noise_scale = 0.000



less_noise_data = make_dataset(args = less_noise_args) # Make the dataset 
less_noise_data['x'] = torch.Tensor(less_noise_data['x']).unsqueeze(1)
less_noise_data['x_test'] = torch.Tensor(less_noise_data['x_test']).unsqueeze(1)

print(less_noise_args.corr_noise_scale, less_noise_args.iid_noise_scale)
print(less_noise_data['x'].shape, less_noise_data['y'].shape, less_noise_data['t'].shape)

0.0 0.0
torch.Size([4000, 1, 40]) (4000,) (40,)


In [ ]:
less_noise_model = get_model_args(as_dict=False)
less_noise_model.device = 'mps'
less_noise_model.total_steps = 6000

print("Custom Conv1d_NN")
results =train_model(less_noise_data, model, less_noise_model)
print()

Custom Conv1d_NN
step 1000, dt 20.48s, train_loss 4.620e-01, test_loss 9.579e-01, train_acc 88.5, test_acc 72.1
step 2000, dt 16.90s, train_loss 2.041e-01, test_loss 9.293e-01, train_acc 91.5, test_acc 77.4
step 3000, dt 16.07s, train_loss 3.565e-01, test_loss 1.039e+00, train_acc 88.0, test_acc 73.5
step 4000, dt 16.19s, train_loss 2.824e-01, test_loss 9.296e-01, train_acc 90.7, test_acc 75.4
step 5000, dt 16.32s, train_loss 1.998e-01, test_loss 9.712e-01, train_acc 93.3, test_acc 75.2
step 6000, dt 16.26s, train_loss 1.677e-01, test_loss 8.623e-01, train_acc 92.8, test_acc 76.5



### Try with more neighbors

In [ ]:
# 1 Nearest Neighbour 
LN_1_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 1, stride=1),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 1, stride=1),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 1, stride=1),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_1_NN_results = train_model(less_noise_data, LN_1_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 15.95s, train_loss 7.050e-01, test_loss 8.853e-01, train_acc 71.7, test_acc 62.7
step 2000, dt 14.73s, train_loss 6.554e-01, test_loss 7.885e-01, train_acc 76.4, test_acc 68.4
step 3000, dt 14.88s, train_loss 6.499e-01, test_loss 7.803e-01, train_acc 78.2, test_acc 67.4
step 4000, dt 15.09s, train_loss 5.422e-01, test_loss 7.817e-01, train_acc 80.4, test_acc 68.5
step 5000, dt 14.92s, train_loss 6.419e-01, test_loss 8.007e-01, train_acc 78.7, test_acc 68.5
step 6000, dt 14.75s, train_loss 5.104e-01, test_loss 7.938e-01, train_acc 81.2, test_acc 68.2



In [ ]:
# 2 Nearby Neighbors
LN_2_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 2, stride=2),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 2, stride=2),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 2, stride=2),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_2_NN_results = train_model(less_noise_data, LN_2_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 16.18s, train_loss 6.143e-01, test_loss 6.880e-01, train_acc 82.2, test_acc 75.4
step 2000, dt 15.13s, train_loss 4.802e-01, test_loss 6.682e-01, train_acc 84.4, test_acc 76.2
step 3000, dt 15.00s, train_loss 5.199e-01, test_loss 6.288e-01, train_acc 87.6, test_acc 78.1
step 4000, dt 15.09s, train_loss 3.524e-01, test_loss 6.767e-01, train_acc 88.5, test_acc 76.8
step 5000, dt 15.08s, train_loss 4.712e-01, test_loss 7.476e-01, train_acc 88.6, test_acc 75.3
step 6000, dt 14.89s, train_loss 3.783e-01, test_loss 7.438e-01, train_acc 88.3, test_acc 75.9



In [ ]:
# 3 Nearest Neighbors
LN_3_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 3, stride=3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3, stride=3),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 3, stride=3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_3_NN_results = train_model(less_noise_data, LN_3_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 15.93s, train_loss 5.379e-01, test_loss 5.674e-01, train_acc 86.8, test_acc 77.6
step 2000, dt 15.16s, train_loss 4.409e-01, test_loss 6.636e-01, train_acc 87.2, test_acc 76.7
step 3000, dt 15.21s, train_loss 4.676e-01, test_loss 7.538e-01, train_acc 89.3, test_acc 78.1
step 4000, dt 15.31s, train_loss 2.744e-01, test_loss 7.043e-01, train_acc 91.6, test_acc 80.6
step 5000, dt 15.41s, train_loss 4.585e-01, test_loss 9.683e-01, train_acc 86.8, test_acc 75.5
step 6000, dt 15.45s, train_loss 3.838e-01, test_loss 7.532e-01, train_acc 92.6, test_acc 79.3



In [ ]:
# 5 Nearest Neighbors
LN_5_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 5, stride=5),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 5, stride=5),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 5, stride=5),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_5_NN_results = train_model(less_noise_data, LN_5_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 16.62s, train_loss 3.737e-01, test_loss 7.674e-01, train_acc 82.7, test_acc 72.8
step 2000, dt 16.15s, train_loss 1.245e-01, test_loss 6.153e-01, train_acc 92.9, test_acc 78.9
step 3000, dt 19.37s, train_loss 3.034e-01, test_loss 7.824e-01, train_acc 88.6, test_acc 76.5
step 4000, dt 18.17s, train_loss 3.574e-01, test_loss 6.591e-01, train_acc 93.4, test_acc 82.1
step 5000, dt 18.98s, train_loss 1.621e-01, test_loss 7.076e-01, train_acc 93.5, test_acc 79.8
step 6000, dt 19.38s, train_loss 7.951e-02, test_loss 7.962e-01, train_acc 93.2, test_acc 81.1



In [ ]:
# 8 Neighbors Conv1d_NN
LN_8_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 8, stride=8),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 8, stride=8),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 8, stride=8),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_8_NN_results = train_model(less_noise_data, LN_8_NN_model, less_noise_model)
print()



MNIST1D Model
step 1000, dt 21.66s, train_loss 5.106e-01, test_loss 7.181e-01, train_acc 84.2, test_acc 74.2
step 2000, dt 20.24s, train_loss 3.568e-01, test_loss 7.086e-01, train_acc 87.7, test_acc 76.8
step 3000, dt 21.14s, train_loss 3.166e-01, test_loss 7.056e-01, train_acc 89.4, test_acc 79.3
step 4000, dt 21.14s, train_loss 3.055e-01, test_loss 8.528e-01, train_acc 87.8, test_acc 78.4
step 5000, dt 20.21s, train_loss 2.606e-01, test_loss 8.664e-01, train_acc 90.3, test_acc 78.5
step 6000, dt 22.14s, train_loss 1.691e-01, test_loss 8.443e-01, train_acc 90.3, test_acc 80.2



In [ ]:
# 12 Neighbors Conv1d_NN
LN_12_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 12, stride=12),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 12, stride=12),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 12, stride=12),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_12_NN_results= train_model(less_noise_data, LN_12_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 20.18s, train_loss 4.412e-01, test_loss 8.427e-01, train_acc 78.1, test_acc 71.2
step 2000, dt 19.42s, train_loss 4.779e-01, test_loss 8.643e-01, train_acc 81.6, test_acc 73.0
step 3000, dt 19.22s, train_loss 3.097e-01, test_loss 6.307e-01, train_acc 87.3, test_acc 77.6
step 4000, dt 19.27s, train_loss 3.508e-01, test_loss 8.706e-01, train_acc 86.4, test_acc 73.7
step 5000, dt 19.31s, train_loss 3.286e-01, test_loss 7.269e-01, train_acc 90.7, test_acc 78.0
step 6000, dt 20.55s, train_loss 3.014e-01, test_loss 7.669e-01, train_acc 89.8, test_acc 77.2



### Best Model with noise data 

In [ ]:
print("MNIST1D Model with 3 Neighbors")
best_model_noise_results = train_model(data, LN_3_NN_model, args)


MNIST1D Model with 3 Neighbors
step 1000, dt 16.50s, train_loss 7.808e-01, test_loss 1.410e+00, train_acc 65.9, test_acc 45.8
step 2000, dt 15.55s, train_loss 7.897e-01, test_loss 1.592e+00, train_acc 70.2, test_acc 47.9
step 3000, dt 15.64s, train_loss 6.783e-01, test_loss 1.953e+00, train_acc 72.7, test_acc 44.4
step 4000, dt 15.89s, train_loss 5.978e-01, test_loss 1.821e+00, train_acc 74.3, test_acc 45.8
step 5000, dt 15.63s, train_loss 5.238e-01, test_loss 2.059e+00, train_acc 76.1, test_acc 45.7
step 6000, dt 17.66s, train_loss 5.332e-01, test_loss 2.220e+00, train_acc 71.5, test_acc 44.1


In [ ]:
print("MNIST1D Model with 5 Neighbors")
best_model_noise_results = train_model(data, LN_5_NN_model, args)

MNIST1D Model with 5 Neighbors
step 1000, dt 20.61s, train_loss 1.026e+00, test_loss 1.385e+00, train_acc 64.0, test_acc 48.2
step 2000, dt 20.93s, train_loss 7.435e-01, test_loss 1.552e+00, train_acc 69.5, test_acc 48.1
step 3000, dt 19.56s, train_loss 7.189e-01, test_loss 1.584e+00, train_acc 70.8, test_acc 47.6
step 4000, dt 19.58s, train_loss 8.023e-01, test_loss 1.707e+00, train_acc 72.2, test_acc 49.5
step 5000, dt 19.53s, train_loss 6.887e-01, test_loss 1.971e+00, train_acc 72.9, test_acc 48.2
step 6000, dt 20.85s, train_loss 5.770e-01, test_loss 1.872e+00, train_acc 73.9, test_acc 46.9


In [ ]:
print("MNIST1D Model with 8 Neighbors")
best_model_noise_results = train_model(data, LN_8_NN_model, args)

MNIST1D Model with 8 Neighbors
step 1000, dt 24.13s, train_loss 1.020e+00, test_loss 1.287e+00, train_acc 59.8, test_acc 50.9
step 2000, dt 22.37s, train_loss 9.920e-01, test_loss 1.409e+00, train_acc 60.8, test_acc 49.3
step 3000, dt 22.20s, train_loss 1.011e+00, test_loss 1.480e+00, train_acc 60.5, test_acc 48.1
step 4000, dt 22.94s, train_loss 9.850e-01, test_loss 1.473e+00, train_acc 63.7, test_acc 48.8
step 5000, dt 23.42s, train_loss 9.235e-01, test_loss 1.589e+00, train_acc 65.0, test_acc 46.3
step 6000, dt 22.75s, train_loss 8.793e-01, test_loss 1.634e+00, train_acc 64.1, test_acc 46.8


In [ ]:
print("MNIST1D Model with 12 Neighbors")
best_model_noise_results = train_model(data, LN_12_NN_model, args)

MNIST1D Model with 12 Neighbors
step 1000, dt 20.13s, train_loss 1.023e+00, test_loss 1.425e+00, train_acc 60.8, test_acc 48.3
step 2000, dt 22.45s, train_loss 1.041e+00, test_loss 1.410e+00, train_acc 62.5, test_acc 47.5
step 3000, dt 21.10s, train_loss 9.421e-01, test_loss 1.537e+00, train_acc 63.3, test_acc 47.3
step 4000, dt 20.32s, train_loss 9.160e-01, test_loss 1.480e+00, train_acc 65.8, test_acc 48.3
step 5000, dt 20.85s, train_loss 9.226e-01, test_loss 1.564e+00, train_acc 65.2, test_acc 47.6
step 6000, dt 20.21s, train_loss 9.036e-01, test_loss 1.783e+00, train_acc 64.3, test_acc 47.9


## ** more neighbors doesnt mean better results (), but there is a middle ground that is better than others. More neighbors -> lower train_acc, but test_acc stays relatively same

### Custom Conv1d_NN same number of parameters 

In [ ]:
# Adjusted Conv1d_NN
adjusted_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('cpu')

# Get summary 
from torchsummary import summary
summary(adjusted_model, (1, 40))




----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 40]             100
         Conv1d_NN-2               [-1, 25, 40]               0
              ReLU-3               [-1, 25, 40]               0
            Conv1d-4               [-1, 25, 40]           1,900
         Conv1d_NN-5               [-1, 25, 40]               0
              ReLU-6               [-1, 25, 40]               0
            Conv1d-7               [-1, 25, 40]           1,900
         Conv1d_NN-8               [-1, 25, 40]               0
              ReLU-9               [-1, 25, 40]               0
          Flatten-10                 [-1, 1000]               0
           Linear-11                   [-1, 10]          10,010
Total params: 13,910
Trainable params: 13,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [ ]:
# Sequential version of MNIST model 
model1 = nn.Sequential(
   nn.Conv1d(1, 25, 5, 2, 1), 
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(125, 10)
)
print(model1.parameters())
# Get summary 
from torchsummary import summary
summary(model1, (1, 40))


<generator object Module.parameters at 0x30774f920>
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
              ReLU-2               [-1, 25, 19]               0
            Conv1d-3               [-1, 25, 10]           1,900
              ReLU-4               [-1, 25, 10]               0
            Conv1d-5                [-1, 25, 5]           1,900
              ReLU-6                [-1, 25, 5]               0
           Flatten-7                  [-1, 125]               0
            Linear-8                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
-----------------------------------------------------------

In [ ]:
print("hello")

hello
